In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
from sklearn.model_selection import train_test_split

In [69]:
#just use one csv as an example
filepath = "/wsu/home/groups/eeg/eeg-data/csv/eeg*.csv"
files = glob.glob(filepath)
file = files[0]
for file in sorted(files):
    print(file.split("/")[-1])

eeg1.csv
eeg10.csv
eeg11.csv
eeg12.csv
eeg13.csv
eeg14.csv
eeg15.csv
eeg16.csv
eeg17.csv
eeg18.csv
eeg19.csv
eeg2.csv
eeg20.csv
eeg21.csv
eeg22.csv
eeg23.csv
eeg24.csv
eeg25.csv
eeg26.csv
eeg27.csv
eeg28.csv
eeg29.csv
eeg3.csv
eeg30.csv
eeg31.csv
eeg32.csv
eeg33.csv
eeg34.csv
eeg35.csv
eeg36.csv
eeg37.csv
eeg38.csv
eeg39.csv
eeg4.csv
eeg40.csv
eeg41.csv
eeg42.csv
eeg43.csv
eeg44.csv
eeg45.csv
eeg46.csv
eeg47.csv
eeg48.csv
eeg49.csv
eeg5.csv
eeg50.csv
eeg51.csv
eeg52.csv
eeg53.csv
eeg54.csv
eeg55.csv
eeg56.csv
eeg57.csv
eeg58.csv
eeg59.csv
eeg6.csv
eeg60.csv
eeg61.csv


In [70]:
#get column information
df = pd.read_csv(file)
print(df.columns)
info_cols = ["time", "label_0", "label_1", "label_2", "Resp Effort-REF", "ECG EKG-REF"]
label_col = ["label_0", "label_1", "label_2"]
data_cols = list(set(df.columns).difference(info_cols))
cols = df[data_cols].columns

Index(['time', 'EEG Fp1-Ref', 'EEG Fp2-Ref', 'EEG F3-Ref', 'EEG F4-Ref',
       'EEG F7-Ref', 'EEG F8-Ref', 'EEG Fz-Ref', 'EEG C3-Ref', 'EEG C4-Ref',
       'EEG Cz-Ref', 'EEG T3-Ref', 'EEG T5-Ref', 'EEG T4-Ref', 'EEG T6-Ref',
       'EEG P3-Ref', 'EEG P4-Ref', 'EEG Pz-Ref', 'EEG O1-Ref', 'EEG O2-Ref',
       'ECG EKG', 'Resp Effort', 'label_0', 'label_1', 'label_2'],
      dtype='object')


In [35]:
#function to process the csv files and extract the labeled sequences
def process_data(file):
    try:
        #load dataframe and set index
        df = pd.read_csv(file)
        df.columns = ['time', 'EEG Fp1-REF', 'EEG Fp2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG Fz-REF', 'EEG Cz-REF', 'EEG Pz-REF',
       'ECG EKG-REF', 'Resp Effort-REF', 'label_0', 'label_1', 'label_2']
        
        df.set_index("time", inplace=True)
        df.index = df.index.astype("timedelta64[ns]")

        #extract numpy arrays
        X = df[data_cols].values
        y = df[label_col].values

        #find find where all annotations agree and make new column for label
        max_val = []
        labels = []
        for row in X:
            max_val.append(max(row))

        for row in y:
            if sum(row) == 3:
                labels.append(1)
            else:
                labels.append(0)
        df['label'] = labels
        df['max'] = max_val

        X = df[data_cols].values
        y = df.label.values
        
        #extract discrete sequences of data labeled as 1s and 0s
        state = 0
        seq = []
        master = []
        labels = []
        for feature,label in zip(X,y):
            if label==state:
                seq.append(feature)
            else:
                if state==0:
                    master.append(seq)
                    labels.append(state)
                else:
                    master.append(seq)
                    labels.append(state)
                state = label
                seq = []

        #only keep sequences equal to or greater than 10 seconds in length
        X = []
        y = []
        for seq,label in zip(master, labels):
            seconds = len(seq)//1000
            if seconds>=10:
                X.append(seq)
                y.append(label)

        X = np.array(X)
        y = np.array(y)

        return(X, y)
    except Exception as e:
        print(str("ERROR WITH FILE {}, {}").format(file, e))
        

In [37]:
#test with one file
X1,y1 = process_data(files[0])

In [38]:
#process all the files and concatenate the result to a master array
X_master, y_master = X1,y1
for f in files:
    if f == file:
        print(str("skipping {}, already processed in the cell above").format(f))
    else:
        try:
            print(f)
            X,y = process_data(f)
            X_master = np.concatenate((X, X_master), axis=0)
            y_master = np.concatenate((y, y_master), axis=0)
        except TypeError:
            print("error with file ", f)

skipping /wsu/home/groups/eeg/eeg-data/csv/eeg1.csv, already processed in the cell above
/wsu/home/groups/eeg/eeg-data/csv/eeg10.csv
/wsu/home/groups/eeg/eeg-data/csv/eeg11.csv
/wsu/home/groups/eeg/eeg-data/csv/eeg12.csv
/wsu/home/groups/eeg/eeg-data/csv/eeg13.csv
/wsu/home/groups/eeg/eeg-data/csv/eeg14.csv
/wsu/home/groups/eeg/eeg-data/csv/eeg15.csv
/wsu/home/groups/eeg/eeg-data/csv/eeg16.csv
/wsu/home/groups/eeg/eeg-data/csv/eeg17.csv


In [54]:
#split array into 10 second chunks
X,y = [],[]
for feature,label in zip(X_master,y_master):
    x_ = np.array_split(feature, 10*256) #10 second chunks
    for elm in x_:
        X.append(elm)
        y.append(label)

In [61]:
#split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [65]:
#save numpy array to disk
np.save("data/X_train.npy", X_train)
np.save("data/y_train.npy", y_train)
np.save("data/X_test.npy", X_test)
np.save("data/y_test.npy", y_test)